In [1]:
import numpy as np

In [12]:
class K_Filter:
    def __init__(self, y0, dt = 1/25):
        '''
            Ak = constant
            xh = [xh_k+1|k, xh_k|k]
            P = [P_k+1|k, P_k|k]
        '''
        self.dt = dt
        self.Ak = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0]])
        self.xh = [None, np.dot(self.inv(np.dot(self.Ak.transpose(),self.Ak)),np.dot(self.Ak.transpose(), y0))] 
        self.P = [None, self.inv(np.dot(self.Ak.transpose(),self.Ak))]
        
        
    def inv(self, A):
        return np.linalg.inv(A)
    
    def state_update_extrap(self):
        
        self.Fk = np.array([[1, 0, np.cos(self.xh[1][3])*self.dt, 0, 0],
                            [0, 1, np.sin(self.xh[1][3])*self.dt, 0, 0],
                            [0, 0, 1, 0, 0],
                            [0, 0, 0, 1, self.dt],
                            [0, 0, 0, 0, 1]])
        self.xh[0] = np.dot(self.Fk, self.xh[1])
        
    def info_mtx_extrap(self):
        self.P[0] = np.dot(self.Fk, np.dot(self.P[1], self.Fk.transpose())) + np.eye(5)
        
    def kalman_gain(self):
        a = np.dot(self.Ak, np.dot(self.P[0], self.Ak.transpose())) + np.eye(2)
        self.KG = np.dot(self.P[0], np.dot(self.Ak.transpose(), self.inv(a)))
        
    def state_update(self, yk1):
        self.xh[1] = self.xh[0] + np.dot(self.KG, (yk1 - np.dot(self.Ak, self.xh[0])))
        
    def info_mtx_update(self):
        self.P[1] = np.dot(np.eye(2) - np.dot(self.KG, self.Ak), self.P[0])
        
    def step_forward(self, yk1 = False, dt = 1/25):
        self.dt = dt
        self.state_update_extrap()
        self.info_mtx_extrap()
        self.kalman_gain()
        
        if yk1 != False:
            self.state_update(yk1)
            self.info_mtx_update(yk1)
            
        self.xh[1] = self.xh[0]
        self.P[1] = self.P[0]
        
        return self.xh[1]

In [14]:
class K_Filter_3O:
    def __init__(self, y0, dt = 1/25):
        '''
            Ak = constant
            xh = [xh_k+1|k, xh_k|k]
            P = [P_k+1|k, P_k|k]
        '''
        self.dt = dt

        self.H = np.eye(6)
        self.xh = np.array([y0[0], y0[1], 1, 1, .1, .1])
        deltas = np.array([21, 21, 5.2, 4.7, 1.2, 1.2])
        self.P = np.dot(deltas.transpose(),deltas)

        self.F = np.array([[1, 0, self.dt, 0, .5*self.dt**2, 0],
                            [0, 1, 0, self.dt, 0, .5*self.dt**2],
                            [0, 0, 1, 0, self.dt, 0],
                            [0, 0, 0, 1, 0, self.dt],
                            [0, 0, 0, 0, 1, 0],
                            [0, 0, 0, 0, 0, 1]])
        self.R = 1.1*self.P
        self.acclim = 15 #m/s^2
        self.Q = np.diag([1, 1, 1, 1, 5, 5])
        
    def step(self, y = None, dt = 1/25):
        self.dt = dt
        self.predict_state()
        self.kalman_gain()
        
        if type(y) != type(None):
            self.current_state(y)
        
        self.process_cov_update()
        self.limit_acceleration()
        return self.xh
        
    def predict_state(self):
        self.xh = np.dot(self.F, self.xh)
        self.P = np.dot(self.F, np.dot(self.P, self.F.transpose()))
        
    def kalman_gain(self):
        A = np.dot(self.P, self.H.transpose())
        B = np.dot(self.H, np.dot(self.P, self.H.transpose())) + self.R
        self.KG = A/B
        
    def current_state(self, y):
        self.xh = self.xh + np.dot(self.KG, (y - np.dot(self.H, self.xh)))
        
    def process_cov_update(self):
        A = (np.eye(6) - np.dot(self.KG,self.H))
        self.P = np.dot(A, self.P)
        
    def limit_acceleration(self):
        self.xh[4:] = np.clip(self.xh[4:],-13,13)